<a href="https://colab.research.google.com/github/iampauladdai/AnomalyDetection/blob/main/GreyKitResearchV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install greykite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime

import numpy as np
import pandas as pd
import plotly


from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_holiday import SilverkiteHoliday
from greykite.algo.forecast.silverkite.constants.silverkite_seasonality import SilverkiteSeasonalityEnum
from greykite.algo.forecast.silverkite.forecast_simple_silverkite_helper import cols_interact
from greykite.common import constants as cst
from greykite.common.features.timeseries_features import build_time_features_df
from greykite.common.features.timeseries_features import convert_date_to_continuous_time
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
df = pd.read_csv('/content/art_daily_flatmiddle peak 2.csv')
df['value'] = df['value'].interpolate()
percentileValue = df['value'].quantile(0.99) #remove anomaly data
df['actuals_unfiltered'] = df['value'][:3076]

In [ ]:
import plotly.express as px
fig = px.line(df, x="timestamp", y=['value'], title='Plot')
fig.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the seasonal data into a pandas DataFrame
data = pd.read_csv('/content/art_daily_flatmiddle peak 2.csv')

# Assuming the data has a 'date' column and a 'value' column
data['date'] = pd.to_datetime(data['timestamp'])
data.set_index('date', inplace=True)

# Define a threshold or percentile to separate the top and bottom values
threshold = data['value'].quantile(0.65)  # Adjust the percentile as desired

# Separate the data into top and bottom groups based on the threshold
top_values = data[data['value'] >= threshold]
bottom_values = data[data['value'] < threshold]

# Plot the original data with top and bottom values
plt.figure(figsize=(10, 6))
plt.plot(data['value'], label='Original Data')
plt.scatter(top_values.index, top_values['value'], color='red', marker='o', label='Top Values')
plt.scatter(bottom_values.index, bottom_values['value'], color='green', marker='o', label='Bottom Values')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Seasonal Data with Top and Bottom Values')
plt.show()
plt.show()


In [ ]:
percentileFilter = bottom_values['value'].quantile(1.00)

In [ ]:
import plotly.express as px
fig = px.scatter(bottom_values, y=['value'], title='Before filtering')
fig.show()

In [ ]:
bottom_values.loc[bottom_values['value'] >= percentileFilter, 'value'] = np.nan

In [ ]:
bottom_values['value'] = bottom_values['value'].interpolate()

<ipython-input-74-70d92ff9da72>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import plotly.express as px
fig = px.scatter(bottom_values, y=['value'], title='After filtering')
fig.show()

In [ ]:
combined_data = pd.concat([top_values, bottom_values], axis=0)

In [ ]:
fig = px.scatter(combined_data, y=['value'], title='After filtering')
fig.show()

In [ ]:
for index in range(len(df['value'])):
    if df['value'][index] > percentileValue:
      df['value'][index] = percentileValue
    if index > 2875:
      df = df.drop(index)

<ipython-input-5-e75ca6504fdc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e75ca6504fdc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e75ca6504fdc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-5-e75ca6504fdc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
import plotly.express as px
df['MA336'] = df['value'].rolling(5).mean()
fig = px.line(df, x="timestamp", y=['value','MA336'], title='Plot')
fig.show()

In [ ]:
def bound_between(min_val, val, max_val):
    """Bound value between min and max."""
    return min(max(val, min_val), max_val)

In [ ]:
# Specifies dataset information
metadata = MetadataParam(
    time_col="timestamp",  # name of the time column
    value_col="value",  # name of the value column
    #freq="H"  # "H" for hourly, "D" for daily, "W" for weekly, etc.
)
from greykite.framework.templates.forecaster import Forecaster

forecaster =  Forecaster()
result = forecaster.run_forecast_config(
    df=df,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=200,  # forecasts 200 steps ahead
        coverage=0.95,  # 95% prediction intervals
        metadata_param=metadata
    )
)

/usr/local/lib/python3.10/dist-packages/greykite/common/time_properties.py:418: UserWarning:

`train_end_date` is not provided, or value column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2014-04-10 23:35:00).

/usr/local/lib/python3.10/dist-packages/greykite/common/time_properties.py:418: UserWarning:

`train_end_date` is not provided, or value column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2014-04-10 23:35:00).



Fitting 3 folds for each of 1 candidates, totalling 3 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_ridge.py:2000: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.10/dist-packages/greykite/algo/changepoint/adalasso/changepoints_utils.py:667: UserWarning:

With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning:

Coordinate descent with no regularization may lead to unexpected results and is discouraged.

/usr/local/lib/python3.10/dist-packages/greykite/algo/forecast/silverkite/forecast_silverkite.py:1142: UserWarning:

The autoregression lags data had to be interpolated at predict time.`past_df` was either not passed to `predict_silverkite` or it was not long enough to calculate all necessery lags which is equal to `max_lag_order`=6048

/usr/local/lib/python3.10/dist-packages/greykite/algo/uncertainty/conditional/conf_interval.py:279: User

In [ ]:
import pickle

# save the model to disk
with open("/content/model.pickle", "wb") as f:
    pickle.dump(model, f)

# load model
#with open("model.pickle", "rb") as f:
   # model = pickle.load(f)

In [ ]:
forecast = result.forecast
fig = forecast.plot()
plotly.io.show(fig)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go


df['forecast'] = forecast.df['forecast']
df['upper'] = forecast.df['forecast_upper']
df['lower'] = forecast.df['forecast_lower']
df['forecast_ts'] = forecast.df['ts']


In [ ]:
ci_boundary_curve_color="rgba(0, 90, 181, 0.5)"  # light blue
ci_boundary_curve_width=0.0 # no line
ci_band_color="rgba(0, 90, 181, 0.15)" # light blue
actual_mode="lines+markers"
actual_points_color="rgba(250, 43, 20, 0.7)"  # red
actual_points_size=2.0
actual_color_opacity=1.0
forecast_curve_color="rgba(0, 90, 181, 0.7)" # blue
forecast_curve_dash="solid"

data = []

fill_dict = {
      "mode": "lines",
      "fillcolor": ci_band_color,
      "fill": "tonexty"
}

lower_bound = go.Scatter(
            name="Lower Bound",
            x=df["forecast_ts"],
            y=df["lower"],
            mode="lines",
            line=dict(
                width=ci_boundary_curve_width,
                color=ci_boundary_curve_color),
            legendgroup="interval"  # show/hide with the upper bound
        )
data.append(lower_bound)

forecast_fill_dict = fill_dict if df['lower'] is not None else {}

upper_bound = go.Scatter(
            name="Upper Bound",
            x=df["forecast_ts"],
            y=df["upper"],
            line=dict(
                width=ci_boundary_curve_width,
                color=ci_boundary_curve_color),
            legendgroup="interval"  # show/hide with the lower bound
            , **forecast_fill_dict)
data.append(upper_bound)

actual_params = {}
if "lines" in actual_mode:
    actual_params.update(line=dict(color=actual_points_color))
if "markers" in actual_mode:
    actual_params.update(marker=dict(color=actual_points_color, size=actual_points_size))
actual = go.Scatter(
    name="Actual",
    x=df['forecast_ts'],
    y=df['actuals_unfiltered'],
    mode=actual_mode,
    opacity=actual_color_opacity,
    **actual_params
)

data.append(actual)

forecast_fill_dict = fill_dict if (df['lower'] is None) != (df['upper'] is None) else {}
forecast = go.Scatter(
    name="Forecast",
    x=df['forecast_ts'],
    y=df['forecast'],
    line=dict(
        color=forecast_curve_color,
        dash=forecast_curve_dash),
    **forecast_fill_dict)
data.append(forecast)

layout = go.Layout(
    xaxis=dict(title="Timestamp"),
    yaxis=dict(title="Values"),
    title=None,
    title_x=0.5,
    showlegend=True,
    # legend order from top to bottom: Actual, Forecast, Upper Bound, Lower Bound
    legend={'traceorder': 'reversed'}
)
fig = go.Figure(data=data, layout=layout)
fig.update()

In [ ]:
#calculate severity
def getSeverity(value,isLower):
  if isLower == False:
    meanData = forecast.df['forecast_upper'].mean()
    stdData = df['actuals_unfiltered'].std() #standard deviation of the anomaly data
    zScore = (value - meanData)/stdData
  else:
    meanData = forecast.df['forecast_lower'].mean() #use when value is less than corresponding lower bound
    stdData = df['actuals_unfiltered'].std() #standard deviation of the anomaly data
    zScore = (meanData - value)/stdData

  severity = zScore * 100/3

  return bound_between(0,severity,100)


In [ ]:
upperBound = -23
lowerBound = -28
actual = 40
dataList = [upperBound,lowerBound]
inBoundary = True

if actual <= upperBound and actual >= lowerBound:
  print('In boundary')

else:
  print('Not boundary')
  inBoundary = False

Not boundary


In [ ]:
def closest_value(input_list, input_value):

  arr = np.asarray(input_list)

  i = (np.abs(arr - input_value)).argmin()

  return arr[i]

In [ ]:
if inBoundary == False:
    forecastValue = closest_value(dataList,actual)
    #Scale to positive value
    if forecastValue < 0:
      actual = actual + abs(forecastValue) - 1
      forecastValue = 1
    elif actual < 0:
      forecastValue = forecastValue + abs(actual) - 1
      actual = 1

    percentageError = (abs(forecastValue-actual)/abs(forecastValue+actual/2)) * 100
    percentageError = bound_between(0,percentageError,100)

    if actual < lowerBound:
      severity = getSeverity(actual,True) #False if greater than upper bound
    else:
      severity = getSeverity(actual,False)

    severityFinal = ((percentageError + severity)/200) * 100
    print(f"Severity: {severity}, Percentage Error: {percentageError}, Severity Final: {severityFinal}")

    if severityFinal >= 52:
      print('Anomaly')
    else:
      print('Not Anomaly')


Severity: 24.964270028801796, Percentage Error: 100, Severity Final: 62.482135014400896
Anomaly


In [ ]:
pd.DataFrame(result.backtest.test_evaluation, index=["Value"]).transpose()  # formats dictionary as a pd.DataFrame

,Value
CORR,0.981912
R2,0.9634
MSE,82.609153
RMSE,9.088958
MAE,5.813678
MedAE,3.758385
MAPE,8617686.528555
MedAPE,7.880059
sMAPE,12.564463
Q80,2.712529


In [ ]:
#readmore: https://nixtla.mintlify.app/statsforecast/index.html